In [12]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [13]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_207897091_322720103", wandb_username="noa-levi")

BASE_METRIC = "accuracy_per_mean_user_and_bot"

In [14]:
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
df = api.get_sweeps_results(["emo2h460"], metric=BASE_METRIC) 

config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
print(config_cols)
print(HPT_cols)


Total number of sweeps: 1
Download sweep_id='emo2h460' data...
['config_seed', 'config_task', 'config_agent', 'config_layers', 'config_dropout', 'config_features', 'config_input_dim', 'config_max_games', 'config_REVIEW_DIM', 'config_hidden_dim', 'config_output_dim', 'config_strategies', 'config_force_train', 'config_ENV_HPT_mode', 'config_architecture', 'config_basic_nature', 'config_total_epochs', 'config_FEATURES_PATH', 'config_bots_per_user', 'config_save_artifacts', 'config_zero_knowledge', 'config_online_sim_type', 'config_use_user_vector', 'config_human_train_size', 'config_loss_weight_type', 'config_ENV_LEARNING_RATE', 'config_personas_balanced', 'config_transformer_nheads', 'config_SIMULATION_EFs_PATH', 'config_OFFLINE_SIM_DATA_PATH', 'config_favorite_topic_method', 'config_personas_group_number', 'config_offline_simulation_size', 'config_simulation_bot_per_user', 'config_simulation_signal_error', 'config_simulation_user_improve', 'config_online_simulation_factor']
['config_see

In [16]:
df[HPT_cols]

,config_seed,config_basic_nature,config_online_simulation_factor
0,5,24,4
1,4,24,4
2,3,24,4
3,2,24,4
4,1,24,4
...,...,...,...
75,5,17,0
76,4,17,0
77,3,17,0
78,2,17,0


In [17]:
numeric_cols = df.select_dtypes(include=np.number).columns
df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

mean_df = grouped.mean()
std_df = grouped.std()

for col in config_cols:
    if col not in mean_df.columns:
        mean_df[col] = df[col]

best_col = mean_df[[c for c in mean_df.columns if (BASE_METRIC in c and BASE_METRIC[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)

best_col

config_basic_nature  config_online_simulation_factor
17                   0                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch5
                     4                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch9
18                   0                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch5
                     4                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch7
19                   0                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch5
                     4                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch8
20                   0                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch5
                     4                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch9
21                   0                                   ENV_Test_accuracy_per_mean_user_and_bot_epoch5
           

In [23]:
def result_metric(sweeps, group_name, drop_list=[0], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric) 

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1 and col != "config_expert_reliability_path"]
    print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    
    # Remove non-numeric columns before computing mean and std
    numeric_cols = df.select_dtypes(include=np.number).columns
    df_numeric = df[list(numeric_cols) + [col for col in HPT_cols if col not in numeric_cols]]

    grouped = df_numeric.groupby([c for c in HPT_cols if c != "config_seed"])

    mean_df = grouped.mean()
    std_df = grouped.std()
    
    # Re-add non-numeric columns before computing best_col
    for col in config_cols:
        if col not in mean_df.columns:
            mean_df[col] = df[col]

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))

    summary_df = pd.DataFrame(df_cols, index=best_col.index)

    ##############################################################################################
    summary_df['CI_length'] = summary_df['CI'].apply(lambda x: x[1] - x[0])
    summary_df['Min_Max'] = summary_df['values'].apply(lambda x: (max(x), min(x)))
    ##############################################################################################

    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    if len(summary_df.index.names) == 1:
        return summary_df.rename_axis(group_name)
    else:
        return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# For HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [19]:
# Create the directory if it doesn't exist
directory = 'sweeps_csvs'
if not os.path.exists(directory):
    os.makedirs(directory)

HPT_results_df = result_metric(["emo2h460"], "option", drop_HPT=False, epoch="best")
HPT_results_df

Total number of sweeps: 1
Download sweep_id='emo2h460' data...
['config_seed', 'config_basic_nature', 'config_online_simulation_factor']


mean       std  \
config_basic_nature config_online_simulation_factor                       
17                  0                                0.789159  0.010007   
                    4                                0.795993  0.012496   
18                  0                                0.789159  0.010007   
                    4                                0.796089  0.012990   
19                  0                                0.789159  0.010007   
                    4                                0.794848  0.003705   
20                  0                                0.789159  0.010007   
                    4                                0.797374  0.009449   
21                  0                                0.789159  0.010007   
                    4                                0.797378  0.011228   
22                  0                                0.789159  0.010007   
                    4                                0.794805  0.004497   
23                  0                                0.789159  0.010007   
                    4                                0.792831  0.011781   
24                  0                                0.789159  0.010007   
                    4                                0.798344  0.011427   

                                                                                                values  \
config_basic_nature config_online_simulation_factor                                                      
17                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.8004488658193148, 0.7910049316881792, 0.813...   
18                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.8030776889613664, 0.7842413216572973, 0.812...   
19                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.7982036627297778, 0.7887782426621658, 0.795...   
20                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.8003521474502342, 0.7867783567635749, 0.810...   
21                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.8062198400532707, 0.7799454924675318, 0.806...   
22                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.7918882783967689, 0.7916566859055123, 0.802...   
23                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.795919630586218, 0.7781349660587329, 0.8099...   
24                  0                                [0.7929287973550296, 0.7818916404961866, 0.801...   
                    4                                [0.8105724201616443, 0.7851809511790032, 0.810...   

                                                     epoch  \
config_basic_nature config_online_simulation_factor          
17                  0                                    5   
                    4                                    9   
18                  0                                    5   
                    4                                    7   
19                  0                                    5   
                    4                                    8   
20                  0                                    5   
                    4                                    9   
21                  0                                    5   
                    4                                    8   
22                  0                                

In [20]:
# sorted(HPT_results_df['mean'], reverse=True)

df_reset = HPT_results_df.copy().reset_index()
new_df = df_reset[['mean', 'std', 'CI_length']]
# new_df = df_reset[['config_basic_nature', 'mean', 'std', 'CI_length']]
#new_df = new_df[new_df['config_basic_nature'].isin([17,18, 19,20,21,22,23,24, 25,26,27,28])]

new_df.sort_values(by='mean', ignore_index=True, ascending=False)



,mean,std,CI_length
0,0.798344,0.011427,0.017041
1,0.797378,0.011228,0.017627
2,0.797374,0.009449,0.014013
3,0.796089,0.012990,0.019135
4,0.795993,0.012496,0.019470
5,0.794848,0.003705,0.006055
6,0.794805,0.004497,0.006816
7,0.792831,0.011781,0.020155
8,0.789159,0.010007,0.015603
9,0.789159,0.010007,0.014974


In [21]:
test_results_df = result_metric(["qbhbd4fu"], "option", drop_HPT=False, epoch="best")
test_results_df

Total number of sweeps: 1
Download sweep_id='qbhbd4fu' data...


['config_seed', 'config_features', 'config_input_dim', 'config_REVIEW_DIM', 'config_FEATURES_PATH', 'config_online_simulation_factor']


mean  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            59               36                data/BERT_PCA_36.csv  0                                0.826854   
                                                                         4                                0.830790   
EFs             60               37                data/EFs_by_GPT35.csv 0                                0.828898   
                                                                         4                                0.836534   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                0.833291   
                                                                         4                                0.832273   

                                                                                                               std  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            59               36                data/BERT_PCA_36.csv  0                                0.002723   
                                                                         4                                0.001577   
EFs             60               37                data/EFs_by_GPT35.csv 0                                0.001633   
                                                                         4                                0.002257   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                0.002611   
                                                                         4                                0.002188   

                                                                                                                                                     values  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                                      
BERT            59               36                data/BERT_PCA_36.csv  0                                [0.8240678419478681, 0.8313392411637519, 0.826...   
                                                                         4                                [0.8296339404406156, 0.832838095337495, 0.8291...   
EFs             60               37                data/EFs_by_GPT35.csv 0                                [0.8277120396253037, 0.8267503772423906, 0.829...   
                                                                         4                                [0.835740803191593, 0.8377435333486027, 0.8396...   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                [0.830059882220964, 0.8358669977445071, 0.8350...   
                                                                         4                                [0.834704839182419, 0.8317728264347153, 0.8294...   

                                                                                                          epoch  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor          
BERT            59               36                data/BERT_PCA_36.csv  0                                    9   
                                                                         4                                   22   
EFs             60               37                data/EFs_by_GPT35.csv 0                                    8   
                                                                         4                                   13   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                    5   
                                                                         4                                    9   

                          

# Result for a specific epoch

In [22]:
sweep_results = result_metric(["qbhbd4fu"], "LLMs", drop_HPT=False, epoch=9)
sweep_results

Total number of sweeps: 1
Download sweep_id='qbhbd4fu' data...
['config_seed', 'config_features', 'config_input_dim', 'config_REVIEW_DIM', 'config_FEATURES_PATH', 'config_online_simulation_factor']


mean  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            59               36                data/BERT_PCA_36.csv  0                                0.826854   
                                                                         4                                0.828760   
EFs             60               37                data/EFs_by_GPT35.csv 0                                0.827504   
                                                                         4                                0.835512   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                0.832466   
                                                                         4                                0.832273   

                                                                                                               std  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor             
BERT            59               36                data/BERT_PCA_36.csv  0                                0.002723   
                                                                         4                                0.002667   
EFs             60               37                data/EFs_by_GPT35.csv 0                                0.002292   
                                                                         4                                0.002512   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                0.004451   
                                                                         4                                0.002188   

                                                                                                                                                     values  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                                      
BERT            59               36                data/BERT_PCA_36.csv  0                                [0.8240678419478681, 0.8313392411637519, 0.826...   
                                                                         4                                [0.8271341506645976, 0.8279450470991481, 0.833...   
EFs             60               37                data/EFs_by_GPT35.csv 0                                [0.8237074781438477, 0.8273519883834847, 0.827...   
                                                                         4                                [0.8364963608632096, 0.8331759866892742, 0.839...   
GPT4            59               36                data/GPT4_PCA_36.csv  0                                [0.8324223004009998, 0.8316471769057168, 0.826...   
                                                                         4                                [0.834704839182419, 0.8317728264347153, 0.8294...   

                                                                                                                                                CI  \
config_features config_input_dim config_REVIEW_DIM config_FEATURES_PATH  config_online_simulation_factor                                             
BERT            59               36                data/BERT_PCA_36.csv  0                                (0.8250439380510753, 0.8292731368738888)   
                                                                         4                                 (0.8270307649549922, 0.831107291642767)   
EFs             60               37                data/EFs_by_GPT35.csv 0                                (0.8255898754417297, 0.8290069143893861)   
                                                                         4                                 (0.833715003962762, 0.8375508778247462)   
GPT4            59               36            